<a href="https://colab.research.google.com/github/tedwards523/ReadME.md/blob/main/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as rq
import bs4
import pandas as pd
import plotly.express as px

In [ ]:
GDPurl = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
page = rq.get(GDPurl, headers=headers)
## print out the first 100 characters just to see what it looks like
page.text[0 : 99]

MGDPurl = 'https://en.wikipedia.org/w/index.php?title=List_of_countries_by_GDP_(nominal)&oldid=1187446467'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
page = rq.get(MGDPurl, headers=headers)
## print out the first 100 characters just to see what it looks like
page.text[0 : 99]

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-l'

In [ ]:
bs4page = bs4.BeautifulSoup(page.text, 'html.parser')
tables = bs4page.find('table',{'class':"wikitable"})

In [ ]:
GDP_tables = pd.read_html(page.text)
GDP = GDP_tables[0]
# The MGDP variable is also assigned here from a specific table, but it's important to
# confirm which table index is correct for MGDP as well after reading all tables.
# For now, I will comment it out to focus on fixing GDP.
# MGDP_tables = pd.read_html(page.text)
# MGDP = MGDP_tables[x] # Replace x with the correct table index


/tmp/ipython-input-4119422798.py:1: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Filling in Missing Data

In [ ]:
def fill_missing(df):
    df["gdpurl"] = df["imf"].fillna(df["worldbank"]).fillna(df["un"])
    return df

Gatting One Table

In [ ]:
GDP = GDP_tables[2]
GDP = GDP.dropna()
GDP.head()

Country/Territory UN region IMF[1][13]            World Bank[14]             \
  Country/Territory UN region   Forecast       Year       Estimate       Year   
0             World         —  104476432       2023      100562011       2022   
1     United States  Americas   26949643       2023       25462700       2022   
2             China      Asia   17700899  [n 1]2023       17963171  [n 3]2022   
3           Germany    Europe    4429838       2023        4072192       2022   
4             Japan      Asia    4230862       2023        4231141       2022   

  United Nations[15]             
            Estimate       Year  
0           96698005       2021  
1           23315081       2021  
2           17734131  [n 1]2021  
3            4259935       2021  
4            4940878       2021

Interactive Plot

In [ ]:
fig = px.bar(df, x="Region", y="IMF", color="Country",title="Stack countries")

fig.show()

Sankey Webpage:                                                    https://github.com/tedwards523/Homework5-Sankey_Diagram-